# Visualization of Coincident ICESat-2, Sentinel, and Argo Float Data

ICESat-2 hackweek final project 
June 16, 2020  
Nicole Abib

In [1]:
import os
from icepyx import icesat2data as ipd
import rasterio
from rasterio.plot import show
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py 
import sys  
sys.path.insert(0, '/home/jovyan/leading_to_phytoplankton/scripts')
import readers as rd
import cartopy.crs as ccrs
from astropy.time import Time
import warnings
warnings.filterwarnings('ignore')
# magic function to enable interactive plotting
%matplotlib widget 

In [2]:
# Define path to sample data
data_loc='/home/jovyan/shared/leading_to_phytoplankton/'
argo_fn = data_loc+'floats_highlats.csv'

In [3]:
# Quick check of csv file contents
!head $argo_fn

-64.977,-155.98,2018,1,2,8,57,6.84,0.0041413,-0.0327,34.033,2.4624
-64.977,-155.98,2018,1,2,8,57,10.58,0.0043545,-0.0316,34.033,2.5056
-64.977,-155.98,2018,1,2,8,57,14.94,0.0041301,-0.0307,34.032,2.8152
-64.977,-155.98,2018,1,2,8,57,20.97,0.0042085,-0.0785,34.036,2.628
-64.977,-155.98,2018,1,2,8,57,26.04,0.0045563,-0.1061,34.037,3.0096
-64.977,-155.98,2018,1,2,8,57,31.18,0.0047467,-0.2544,34.042,3.5136
-64.977,-155.98,2018,1,2,8,57,35.67,0.0033291,-1.6271,34.186,3.1104
-64.977,-155.98,2018,1,2,8,57,40.21,0.0029811,-1.703,34.196,3.5064
-64.977,-155.98,2018,1,2,8,57,45.24,0.0036319,-1.6948,34.2,3.708
-64.977,-155.98,2018,1,2,8,57,50.29,0.0020383,-1.7466,34.206,2.412


In [4]:
# Load the csv file with Pandas
argo_df = pd.read_csv(argo_fn)
# Add column names defined in the metadata
argo_df.columns = ['lat', 'lon', 'year', 'month', 'day', 'hour', 
                   'minute', 'Depth (m)', 'bbp (700 nm)', 'temperature', 
                   'salinity', 'chlorophyll']
# Inspect the dataframe
argo_df

,lat,lon,year,month,day,hour,minute,Depth (m),bbp (700 nm),temperature,salinity,chlorophyll
0,-64.977,-155.980,2018,1.0,2.0,8.0,57.0,10.58,0.004354,-0.0316,34.033,2.50560
1,-64.977,-155.980,2018,1.0,2.0,8.0,57.0,14.94,0.004130,-0.0307,34.032,2.81520
2,-64.977,-155.980,2018,1.0,2.0,8.0,57.0,20.97,0.004209,-0.0785,34.036,2.62800
3,-64.977,-155.980,2018,1.0,2.0,8.0,57.0,26.04,0.004556,-0.1061,34.037,3.00960
4,-64.977,-155.980,2018,1.0,2.0,8.0,57.0,31.18,0.004747,-0.2544,34.042,3.51360
...,...,...,...,...,...,...,...,...,...,...,...,...
449725,-50.810,4.359,2019,1.0,2.0,0.0,34.0,134.00,0.002312,2.3460,33.906,0.85765
449726,-50.810,4.359,2019,1.0,2.0,0.0,34.0,136.00,0.001488,2.2490,33.919,0.52873
449727,-50.810,4.359,2019,1.0,2.0,0.0,34.0,138.00,0.001300,2.1780,33.929,0.37503
449728,-50.810,4.359,2019,1.0,2.0,0.0,34.0,140.00,0.001373,2.1080,33.943,0.27205


In [5]:
# Check data types
argo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 449730 entries, 0 to 449729
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   lat           449730 non-null  float64
 1   lon           449730 non-null  float64
 2   year          449730 non-null  int64  
 3   month         449729 non-null  float64
 4   day           449729 non-null  float64
 5   hour          449729 non-null  float64
 6   minute        449729 non-null  float64
 7   Depth (m)     449729 non-null  float64
 8   bbp (700 nm)  449729 non-null  float64
 9   temperature   449729 non-null  float64
 10  salinity      449729 non-null  float64
 11  chlorophyll   449729 non-null  float64
dtypes: float64(11), int64(1)
memory usage: 41.2 MB


In [6]:
# Check that column labels are correct
argo_df.columns

Index(['lat', 'lon', 'year', 'month', 'day', 'hour', 'minute', 'Depth (m)',
       'bbp (700 nm)', 'temperature', 'salinity', 'chlorophyll'],
      dtype='object')

In [7]:
# Check head
argo_df.head()

,lat,lon,year,month,day,hour,minute,Depth (m),bbp (700 nm),temperature,salinity,chlorophyll
0,-64.977,-155.98,2018,1.0,2.0,8.0,57.0,10.58,0.004354,-0.0316,34.033,2.5056
1,-64.977,-155.98,2018,1.0,2.0,8.0,57.0,14.94,0.004130,-0.0307,34.032,2.8152
2,-64.977,-155.98,2018,1.0,2.0,8.0,57.0,20.97,0.004209,-0.0785,34.036,2.6280
3,-64.977,-155.98,2018,1.0,2.0,8.0,57.0,26.04,0.004556,-0.1061,34.037,3.0096
4,-64.977,-155.98,2018,1.0,2.0,8.0,57.0,31.18,0.004747,-0.2544,34.042,3.5136


In [8]:
# Check tail
argo_df.tail()

,lat,lon,year,month,day,hour,minute,Depth (m),bbp (700 nm),temperature,salinity,chlorophyll
449725,-50.81,4.359,2019,1.0,2.0,0.0,34.0,134.0,0.002312,2.346,33.906,0.85765
449726,-50.81,4.359,2019,1.0,2.0,0.0,34.0,136.0,0.001488,2.249,33.919,0.52873
449727,-50.81,4.359,2019,1.0,2.0,0.0,34.0,138.0,0.001300,2.178,33.929,0.37503
449728,-50.81,4.359,2019,1.0,2.0,0.0,34.0,140.0,0.001373,2.108,33.943,0.27205
449729,-50.81,4.359,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
argo_df_nan = argo_df.replace(99999, np.nan)
argo_df = argo_df_nan

In [10]:
# Create a scatter plot showing data locations
ax = argo_df_nan.plot(x='lon', y='lat', kind='scatter', c='k', s=1)
ax.set_title('Location of Argo Floats');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Load in ICESat-2 Data

In [11]:
# temp_dloc = '/home/jovyan/tutorial-data/sea-ice/'
# is2_fn = 'ATL03_20190206123200_06120204_002_01.h5'
is2_fn = 'IS2_S2/ATL03_20190805215948_05930404_002_02.h5'
is2_f = h5py.File(data_loc+is2_fn, 'r')
# is2_f = h5py.File(is2_fn,'r')
print(is2_f['orbit_info/sc_orient'][0]) # Check forward/backward orientation

0


In [12]:
# Read in icesat 2 data as a pandas dataframe
beam='gt1l'
df03=rd.getATL03(is2_f,beam)
df03.head()

,lats,lons,x,y,heights,dt,conf
0,79.993666,40.449280,8938354.0,3306.422607,-2426.635986,5.027759e+07,-2
1,79.993605,40.450261,8938344.0,3289.306152,234.947739,5.027759e+07,2
2,79.993611,40.450256,8938344.0,3289.260986,242.102203,5.027759e+07,2
3,79.993612,40.450246,8938344.0,3289.424561,216.635086,5.027759e+07,2
4,79.993617,40.450247,8938345.0,3289.272705,240.364456,5.027759e+07,2


## Load in Sentinel 2 Data

In [13]:
s2_fn = 'IS2_S2/2019085_T11XMK.tif'
s2_img = rasterio.open(data_loc+s2_fn)

In [14]:
show(s2_img)

In [15]:
s2_bounds  = s2_img.bounds
print(s2_bounds)

BoundingBox(left=399960.0, bottom=8890200.0, right=509760.0, top=9000000.0)


In [16]:
from shapely.geometry import box
geom = box(*s2_bounds)
print(geom.wkt)

POLYGON ((509760 8890200, 509760 9000000, 399960 9000000, 399960 8890200, 509760 8890200))


In [17]:
s2_img.crs

CRS.from_epsg(32611)

In [19]:
geom.exterior.xy

(array('d', [509760.0, 509760.0, 399960.0, 399960.0, 509760.0]),
 array('d', [8890200.0, 9000000.0, 9000000.0, 8890200.0, 8890200.0]))

In [20]:
var= 'heights' #choose which variable we want to plot

## we will want to set colorbar parameters based on the chosen variable
vmin=-10
vmax=30
ticks=np.arange(-20,100,5)

# cartopy - need to make figure and set projection first before adding data
plt.figure(figsize=(8,8), dpi= 90)
ax = plt.axes(projection=ccrs.NorthPolarStereo(central_longitude=0)) # choose polar sterographic for projection
ax.coastlines(resolution='50m', color='black', linewidth=1)
ax.set_extent([-180, 180, 60, 90], ccrs.PlateCarree())
plt.scatter(argo_df['lon'], argo_df['lat'], c= 'k',s=2,transform=ccrs.PlateCarree())
plt.scatter(df03['lons'][::1000], df03['lats'][::1000],c=df03[var][::1000], cmap='viridis', vmin=vmin,vmax=vmax,transform=ccrs.PlateCarree())
plt.plot(*geom.exterior.xy, transform=ccrs.PlateCarree())
# ax.set_extent([399960.0, 8890200.0, 509760.0, 9000000.0], transform=ccrs.PlateCarree())
plt.colorbar(label=var, shrink=0.5, ticks=ticks,extend='both');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# When i have coincident data, I can plot these on top of eachother

In [ ]:
df03.head()

In [ ]:
## create bounding box of interest:
lonmin,lonmax,latmin,latmax= [0,60,85,87.5]

bbox=(df03['lons']>lonmin)&(df03['lons']<lonmax)&(df03['lats']>latmin)&(df03['lats']<latmax)
df_cut=df03[bbox]

## converting GPS time to UTC time
epoch=is2_f['/ancillary_data/atlas_sdp_gps_epoch'][0]
df_cut['time']=Time(epoch+df_cut['dt'],format='gps').utc.datetime

## calculate along track distance relative to the beginning of the cut segment
# Subtract all along track x-values from first segment
df_cut['AT_dist']=df_cut.x-df_cut.x.values[0]

df_cut.head()

In [ ]:
plt.subplots(1,1,figsize=(10,4))
plt.scatter(df_cut.AT_dist.values[df_cut.conf<3],df_cut.heights.values[df_cut.conf<3],c='r',s=1)
plt.scatter(df_cut.AT_dist.values[df_cut.conf>2],df_cut.heights.values[df_cut.conf>2],c='g',s=1)
plt.xlabel('alongtrack distance (m)',fontsize=20)
plt.ylabel('heights',fontsize=20)
# plt.ylim(15,25);
plt.xlim(df_cut.AT_dist.values[0],df_cut.AT_dist.values[-1]);